In [1]:
from sympy.physics.vector import (
    ReferenceFrame,
    dynamicsymbols,
    get_motion_params
)
from sympy import symbols, Piecewise, Eq, solveset, Min, Max, latex, acos, Matrix, eye
from sympy.utilities.lambdify import lambdify

from IPython.display import Markdown, display
import ipywidgets as widgets
from ipywidgets import interactive_output

import numpy as np
import plotly.graph_objects as go

# Reference frame

In [2]:
N = ReferenceFrame('N')
A = ReferenceFrame('A')

In [3]:
Markdown(rf"""
$$N = \{{ {latex(N.x)}, {latex(N.y)}, {latex(N.z)} \}}$$
$$A = \{{ {latex(A.x)}, {latex(A.y)}, {latex(A.z)} \}}$$
""")


$$N = \{ \mathbf{\hat{n}_x}, \mathbf{\hat{n}_y}, \mathbf{\hat{n}_z} \}$$
$$A = \{ \mathbf{\hat{a}_x}, \mathbf{\hat{a}_y}, \mathbf{\hat{a}_z} \}$$


# Reference variables

In [4]:
T = symbols('T', nonnegative=True)
R = symbols('R', nonnegative=True)
t = symbols('t', nonnegative=True)

In [5]:
a_magnitude = symbols('|a|', nonnegative=True)
a_vector = A.x * Piecewise((-a_magnitude, T * R - dynamicsymbols._t < 0), (a_magnitude, True))
# a_vector = A.x * a_magnitude * (T * R - dynamicsymbols._t) / Abs(T * R - dynamicsymbols._t)

 # Since we have no initial velocity, acceleration and velocity will be on the same axis, so velocity can be defined in A Frame
v0_magnitude = symbols('|v_{0}|', nonnegative=True)
vT_magnitude = symbols('|v_{T}|')
v0_vector = A.x * v0_magnitude
vT_vector = A.x * vT_magnitude

p0_x, p0_y, p0_z = symbols('p_{0x} p_{0y} p_{0z}')
pT_x, pT_y, pT_z = symbols('p_{Tx} p_{Ty} p_{Tz}')
p0_vector = N.x * p0_x + N.y * p0_y + N.z * p0_z
pT_vector = N.x * pT_x + N.y * pT_y + N.z * pT_z

direction_vector = pT_vector - p0_vector

In [6]:
Nx = N.x.to_matrix(N)
NA1 = direction_vector.normalize().to_matrix(N)

v = Nx.cross(NA1)
c = Nx.dot(NA1) # cosine of angle
K = Matrix(3, 3, [
    0, - v[2], v[1],
    v[2], 0, - v[0],
    - v[1], v[0], 0
])
rotation_matrix = eye(3) + K + (K * K) * (1 - c) / (1 - c ** 2)

NA2 = (rotation_matrix * N.y.to_matrix(N)).simplify()

dcm = Matrix(3, 3, [*NA1, *NA2, *NA1.cross(NA2)])

A.orient(N, 'DCM', dcm.transpose())

In [7]:
Markdown(rf"""
Direction vector in N: ${latex(direction_vector.to_matrix(N).simplify())}$

Direction vector in A: ${latex(direction_vector.to_matrix(A).simplify())}$
""")


Direction vector in N: $\left[\begin{matrix}- p_{0x} + p_{Tx}\\- p_{0y} + p_{Ty}\\- p_{0z} + p_{Tz}\end{matrix}\right]$

Direction vector in A: $\left[\begin{matrix}\sqrt{\left(p_{0x} - p_{Tx}\right)^{2} + \left(p_{0y} - p_{Ty}\right)^{2} + \left(p_{0z} - p_{Tz}\right)^{2}}\\0\\0\end{matrix}\right]$


In [8]:
Markdown(rf"""
$$
\begin{{align*}}
|a| & \quad & m.s^{{-2}} \quad & \text{{ Acceleration magnitude}} \\
v_0 &= {latex(v0_vector)} \quad & m.s^{{-1}} \quad & \text{{ Velocity at }} t_0 \\
v_T &= {latex(vT_vector)} \quad & m.s^{{-1}} \quad & \text{{ Velocity at destination}} \\
p_0 &= {latex(p0_vector)} \quad & m \quad & \text{{ Position at }} t_0 \\
p_T &= {latex(pT_vector)} \quad & m \quad & \text{{ Position at destination}} \\
direction &= {latex(direction_vector)} \quad & m \quad & \text{{ Direction vector }}
\end{{align*}}
$$
""")


$$
\begin{align*}
|a| & \quad & m.s^{-2} \quad & \text{ Acceleration magnitude} \\
v_0 &= |v_{0}|\mathbf{\hat{a}_x} \quad & m.s^{-1} \quad & \text{ Velocity at } t_0 \\
v_T &= |v_{T}|\mathbf{\hat{a}_x} \quad & m.s^{-1} \quad & \text{ Velocity at destination} \\
p_0 &= p_{0x}\mathbf{\hat{n}_x} + p_{0y}\mathbf{\hat{n}_y} + p_{0z}\mathbf{\hat{n}_z} \quad & m \quad & \text{ Position at } t_0 \\
p_T &= p_{Tx}\mathbf{\hat{n}_x} + p_{Ty}\mathbf{\hat{n}_y} + p_{Tz}\mathbf{\hat{n}_z} \quad & m \quad & \text{ Position at destination} \\
direction &= (- p_{0x} + p_{Tx})\mathbf{\hat{n}_x} + (- p_{0y} + p_{Ty})\mathbf{\hat{n}_y} + (- p_{0z} + p_{Tz})\mathbf{\hat{n}_z} \quad & m \quad & \text{ Direction vector }
\end{align*}
$$


# Motion parameters, with constant acceleration  𝑎

In [9]:
def replace_max_0_RT(f):
    # Given that T >= 0 and R in [0, 1]:
    #  We can say that max(0, R * T) = R * T
    #  However sympy is not able to infer this, so we replace manually
    return f.replace(Max, lambda a, b: (b if a == 0 else a_))

In [10]:
original_a_fn, original_v_fn, original_d_fn = get_motion_params(
    A,
    acceleration=a_vector, 
    velocity=v0_vector,
    position=p0_vector
)

a_fn = original_a_fn.subs(dynamicsymbols._t, t)
v_fn = original_v_fn.subs(dynamicsymbols._t, t)
d_fn = original_d_fn.subs(dynamicsymbols._t, t).applyfunc(replace_max_0_RT)

In [11]:
Markdown(rf"""
These formulae depend on:
* $T$ (in seconds): The total time to arrival
* $R \in [0, 1]$: The ratio at which we start decelerating

$$
\begin{{align}}
a(t) &= {latex(a_fn)} \\
v(t) &= {latex(v_fn)} \\
d(t) &= {latex(d_fn.subs({p0_x: 0, p0_y: 0, p0_z: 0}) + p0_vector)}
\end{{align}}
$$

**Remark**: $d(t)$ is rewritten here as a combination of A and N for brevity
""")


These formulae depend on:
* $T$ (in seconds): The total time to arrival
* $R \in [0, 1]$: The ratio at which we start decelerating

$$
\begin{align}
a(t) &= \begin{cases} - |a| & \text{for}\: R T - t < 0 \\|a| & \text{otherwise} \end{cases}\mathbf{\hat{a}_x} \\
v(t) &= (- t |a| + 2 |a| \min\left(t, R T\right) + |v_{0}|)\mathbf{\hat{a}_x} \\
d(t) &= (- \frac{t^{2} |a|}{2} + t \left(2 R T |a| + |v_{0}|\right) + |a| \min\left(t, R T\right)^{2} + |v_{0}| \min\left(t, R T\right) - \left(2 R T |a| + |v_{0}|\right) \min\left(t, R T\right))\mathbf{\hat{a}_x} + p_{0x}\mathbf{\hat{n}_x} + p_{0y}\mathbf{\hat{n}_y} + p_{0z}\mathbf{\hat{n}_z}
\end{align}
$$

**Remark**: $d(t)$ is rewritten here as a combination of A and N for brevity


# Finding R

In [12]:
def replace_min_T(f):
    # Given that T >= 0 and R in [0, 1]:
    #  We can say that min(t, R * T) = R * T
    #  However sympy is not able to infer this, so we replace manually
    return f.replace(Min(T, R*T), R*T)

In [13]:
eq_vT_x = Eq(
    vT_vector.to_matrix(A)[0],
    v_fn.subs(t, T).applyfunc(replace_min_T).to_matrix(A)[0]
)

solved_R = list(solveset(eq_vT_x, R))[0] # The solveset returns a singleton, which is also a list of size 1

In [14]:
Markdown(rf"""
$R \in [0, 1]$: The ratio at which we start decelerating

$$
\begin{{align}}
v_T &= v(T) \\
{latex(vT_vector.to_matrix(A)[0])} &= {latex(v_fn.subs(t, T).to_matrix(A)[0])} \\
\end{{align}}
$$

This solves into:

$$R = {latex(solved_R)}$$
""")


$R \in [0, 1]$: The ratio at which we start decelerating

$$
\begin{align}
v_T &= v(T) \\
|v_{T}| &= - T |a| + 2 |a| \min\left(T, R T\right) + |v_{0}| \\
\end{align}
$$

This solves into:

$$R = \frac{T |a| - |v_{0}| + |v_{T}|}{2 T |a|}$$


# Finding T

In [15]:
eq_dT_x = Eq(
    pT_vector.to_matrix(A).simplify()[0],
    d_fn.subs({t: T}).applyfunc(replace_min_T).subs({R: solved_R}).to_matrix(A)[0]
)

# eq_dT_x solves as a quadratic expression, thus returns a finite set of two values.
solved_T1, solved_T2 = solveset(eq_dT_x, T)
solved_T = Max(solved_T1, solved_T2)

In [16]:
show_toggle = widgets.Checkbox(
    value=False,
    description='Display T',
    disabled=False,
    indent=False
)

out = widgets.Output(layout=widgets.Layout())

md_content = Markdown(rf"""
$T$ (in seconds): The total time to arrival

$$
\begin{{align}}
p_T &= p(T) \\
{latex(pT_vector.to_matrix(A)[0])} &= {latex(d_fn.subs({t: T}).to_matrix(A)[0])} \\
\\
\text{{Given that }} R &= {latex(solved_R)} \\
{latex(pT_vector.to_matrix(A)[0])} &= {latex(d_fn.subs({t: T, R: solved_R}).to_matrix(A)[0])} \\
\end{{align}}
$$

This solves into:

$$T = {latex(solved_T)}$$
""")

def display_T(displayed):
    with out:
        if displayed['new']:
            display(md_content)
        else:
            out.clear_output()
        
show_toggle.observe(display_T, 'value')

display(show_toggle)
display(out)

Checkbox(value=False, description='Display T', indent=False)

Output()

# Motion parameters, expand equations

In [17]:
expanded_a_fn = a_fn.subs({T: solved_T, R: solved_R})
expanded_v_fn_2 = v_fn.subs({T: solved_T, R: solved_R})
expanded_d_fn_2 = d_fn.subs({T: solved_T, R: solved_R})

In [18]:
show_toggle = widgets.Checkbox(
    value=False,
    description='Display expanded motion parameters',
    disabled=False,
    indent=False
)

out = widgets.Output(layout=widgets.Layout())

md_content = Markdown(rf"""
After injecting $T$ and $R$, we obtain:

$$
\begin{{align}}
a(t) &= {latex(expanded_a_fn)} \\
v(t) &= {latex(expanded_v_fn_2)} \\
d(t) &= {latex(expanded_d_fn_2)}
\end{{align}}
$$
""")

def display_motion_parameters(displayed):
    with out:
        if displayed['new']:
            display(md_content)
        else:
            out.clear_output()
        
show_toggle.observe(display_motion_parameters, 'value')

display(show_toggle)
display(out)

Checkbox(value=False, description='Display expanded motion parameters', indent=False)

Output()

# Plotting

In [19]:
a_knob = widgets.BoundedFloatText(value=1, min=0.1, max=15.0, step=0.1, description='Acceleration (m.s-2)')
v0_knob = widgets.IntText(value=0, step=10, description='v0 (m.s-1)')
vT_knob = widgets.IntText(value=0, step=10, description='vT (m.s-1)')

p0x_knob = widgets.IntText(value=0, step=10, description='p0.x (m)')
p0y_knob = widgets.IntText(value=0, step=10, description='p0.y (m)')
p0z_knob = widgets.IntText(value=0, step=10, description='p0.z (m)')

pTx_knob = widgets.IntText(value=100000, step=10, description='pT.x (m)')
pTy_knob = widgets.IntText(value=0, step=10, description='pT.y (m)')
pTz_knob = widgets.IntText(value=0, step=10, description='pT.z (m)')

ui = widgets.VBox([
    widgets.HBox([a_knob, v0_knob, vT_knob]),
    widgets.HBox([p0x_knob, p0y_knob, p0z_knob]),
    widgets.HBox([pTx_knob, pTy_knob, pTz_knob])
])
display(ui)

In [20]:
def make_plot(title, short_name, fn, args, line_shape="spline"):
    v_args = {a_magnitude: args["a_knob"], v0_magnitude: args["v0_knob"], vT_magnitude: args["vT_knob"]}
    R_val = solved_R.subs(v_args)
    
    p_args = {
        **v_args,
        p0_x: args["p0x_knob"],
        p0_y: args["p0y_knob"],
        p0_z: args["p0z_knob"],
        pT_x: args["pTx_knob"],
        pT_y: args["pTy_knob"],
        pT_z: args["pTz_knob"],
    }
    T_val = solved_T.subs(p_args)

    
    t_sample = np.linspace(0, float(T_val), 99)
    image_sample = lambdify(t, fn.to_matrix(N).subs({**p_args, T: T_val, R: R_val}))
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=list(t_sample), y=[image_sample(t_val)[0][0] for t_val in t_sample], line_shape=line_shape, name=f"{short_name} - x"))
    fig.add_trace(go.Scatter(x=list(t_sample), y=[image_sample(t_val)[1][0] for t_val in t_sample], line_shape=line_shape, name=f"{short_name} - y"))
    fig.add_trace(go.Scatter(x=list(t_sample), y=[image_sample(t_val)[2][0] for t_val in t_sample], line_shape=line_shape, name=f"{short_name} - z"))

    fig.update_layout(title=title, xaxis_title="t", yaxis_title=short_name)
    return fig

def plot_position(**knob_args):
    plot = make_plot("Position", "p(t)", d_fn, knob_args)
    plot.show()
    
def plot_velocity(**knob_args):
    plot = make_plot("Velocity", "v(t)", v_fn, knob_args)
    plot.show()
    
def plot_acceleration(**knob_args):
    plot = make_plot("Acceleration", "a(t)", a_fn, knob_args, line_shape="hv")
    plot.show()

In [21]:
knob_args = {
    "a_knob": a_knob,
    "v0_knob": v0_knob, "vT_knob": vT_knob,
    "p0x_knob": p0x_knob, "p0y_knob": p0y_knob, "p0z_knob": p0z_knob,
    "pTx_knob": pTx_knob, "pTy_knob": pTy_knob, "pTz_knob": pTz_knob
}

position_plot = interactive_output(plot_position, knob_args)
velocity_plot = interactive_output(plot_velocity, knob_args)
acceleration_plot = interactive_output(plot_acceleration, knob_args)
display(position_plot, velocity_plot, acceleration_plot)

Output()

Output()

Output()

In [22]:
knob_args = {
    "a_knob": a_knob,
    "v0_knob": v0_knob, "vT_knob": vT_knob,
    "p0x_knob": p0x_knob, "p0y_knob": p0y_knob, "p0z_knob": p0z_knob,
    "pTx_knob": pTx_knob, "pTy_knob": pTy_knob, "pTz_knob": pTz_knob
}

def plot_3d_position(**args):
    v_args = {a_magnitude: args["a_knob"], v0_magnitude: args["v0_knob"], vT_magnitude: args["vT_knob"]}
    R_val = solved_R.subs(v_args)
    
    p_args = {
        **v_args,
        p0_x: args["p0x_knob"],
        p0_y: args["p0y_knob"],
        p0_z: args["p0z_knob"],
        pT_x: args["pTx_knob"],
        pT_y: args["pTy_knob"],
        pT_z: args["pTz_knob"],
    }
    T_val = solved_T.subs(p_args)

    t_sample = np.linspace(0, float(T_val), 99)
    d = lambdify(t, d_fn.to_matrix(N).subs({**p_args, T: T_val, R: R_val}))
    p_mag = lambdify(t, d_fn.to_matrix(A).subs({**p_args, T: T_val, R: R_val}))
    v = lambdify(t, v_fn.to_matrix(N).subs({**p_args, T: T_val, R: R_val}))
    v_mag = lambdify(t, v_fn.to_matrix(A).subs({**p_args, T: T_val, R: R_val}))
    a = lambdify(t, a_fn.to_matrix(N).subs({**p_args, T: T_val, R: R_val}))
    a_mag = lambdify(t, a_fn.to_matrix(A).subs({**p_args, T: T_val, R: R_val}))

    fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x=[d(t_val)[0][0] for t_val in t_sample],
        y=[d(t_val)[1][0] for t_val in t_sample],
        z=[d(t_val)[2][0] for t_val in t_sample],
        customdata=np.dstack((
            [p_mag(t_val)[0][0] for t_val in t_sample],
            [v_mag(t_val)[0][0] for t_val in t_sample],
            [v(t_val)[0][0] for t_val in t_sample],
            [v(t_val)[1][0] for t_val in t_sample],
            [v(t_val)[2][0] for t_val in t_sample],
            [a_mag(t_val)[0][0] for t_val in t_sample],
            [a(t_val)[0][0] for t_val in t_sample],
            [a(t_val)[1][0] for t_val in t_sample],
            [a(t_val)[2][0] for t_val in t_sample],
        ))[0],
        hovertemplate='''
            p:%{customdata[0]:.1f} m - (%{x:.1f} %{y:.1f} %{z:.1f})<br>
            v:%{customdata[1]:.1f} m/s - (x:%{customdata[2]:.1f}, y:%{customdata[3]:.1f}, z:%{customdata[4]:.1f})<br>
            a:%{customdata[5]:.1f} m/s2 - (x:%{customdata[6]:.1f}, y:%{customdata[7]:.1f}, z:%{customdata[8]:.1f})
        ''',
        line={
            "color": list(t_sample),
            "colorscale": "Viridis",
            "colorbar": { "title": "t" }
        },
        marker={ "size": 2, "color": list(t_sample), "colorscale": "Viridis" },
        name=f"p(t)",

    ))

    fig.update_layout(title="3D Position", xaxis_title="p(x)", yaxis_title="p(y)", autosize=True, height=700)
    fig.show(config={'scrollZoom': False})

position_3d_plot = interactive_output(plot_3d_position, knob_args)
display(position_3d_plot)

Output()